# 🏷️ Notebook 2: Rotulagem e Análise Exploratória de Dados

Este notebook conduz a análise exploratória dos dados coletados e demonstra o processo de rotulagem para criação do dataset de treinamento.

## 🎯 Objetivos
- Carregar e explorar dados coletados
- Realizar análise exploratória (EDA)
- Preparar amostra para rotulagem manual
- Analisar qualidade e distribuição dos dados
- Configurar Label Studio para rotulagem

## 📋 Pré-requisitos
- Dados coletados no Notebook 1
- Label Studio instalado
- Dados pré-processados disponíveis

In [ ]:
# Imports para análise exploratória
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
from datetime import datetime
import re
from collections import Counter
from wordcloud import WordCloud

# Configuração de plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

# Adicionar src ao path
sys.path.append('../src')

# Configurar diretórios
project_root = Path("..").resolve()
data_dir = project_root / "data" 
raw_dir = data_dir / "raw"
processed_dir = data_dir / "processed"

print(f"📁 Diretório do projeto: {project_root}")
print(f"📊 Diretório de dados brutos: {raw_dir}")
print(f"🔧 Diretório de dados processados: {processed_dir}")

# Verificar se existem dados para analisar
if raw_dir.exists():
    raw_folders = [f for f in raw_dir.iterdir() if f.is_dir()]
    print(f"📂 Execuções de coleta disponíveis: {len(raw_folders)}")
    for folder in sorted(raw_folders)[-3:]:  # Mostrar as 3 mais recentes
        files = list(folder.glob("*.csv"))
        print(f"  - {folder.name}: {len(files)} arquivos CSV")
else:
    print("⚠️  Diretório de dados brutos não encontrado")
    print("💡 Execute o Notebook 1 primeiro para coletar dados")

In [ ]:
# Carregar dados da execução mais recente
def load_latest_data():
    """Carrega os dados da execução mais recente"""
    if not raw_dir.exists() or not any(raw_dir.iterdir()):
        return None, None
    
    # Encontrar a pasta mais recente
    latest_folder = max([f for f in raw_dir.iterdir() if f.is_dir()], 
                       key=lambda x: x.name)
    
    print(f"📂 Carregando dados de: {latest_folder.name}")
    
    # Carregar Twitter data
    twitter_file = latest_folder / "twitter_data.csv"
    twitter_df = None
    if twitter_file.exists():
        twitter_df = pd.read_csv(twitter_file)
        twitter_df['platform'] = 'twitter'
        print(f"🐦 Twitter: {len(twitter_df)} posts")
    else:
        print("⚠️  Arquivo do Twitter não encontrado")
    
    # Carregar Reddit data  
    reddit_file = latest_folder / "reddit_data.csv"
    reddit_df = None
    if reddit_file.exists():
        reddit_df = pd.read_csv(reddit_file)
        reddit_df['platform'] = 'reddit'
        print(f"🔴 Reddit: {len(reddit_df)} posts")
    else:
        print("⚠️  Arquivo do Reddit não encontrado")
    
    return twitter_df, reddit_df

# Carregar dados
twitter_df, reddit_df = load_latest_data()

# Combinar dados se ambos existirem
combined_df = None
if twitter_df is not None and reddit_df is not None:
    # Padronizar colunas para combinação
    twitter_columns = {'text': 'text', 'timestamp': 'timestamp', 'platform': 'platform'}
    reddit_columns = {'title': 'text', 'timestamp': 'timestamp', 'platform': 'platform'}
    
    twitter_std = twitter_df[list(twitter_columns.keys())].rename(columns=twitter_columns)
    reddit_std = reddit_df[list(reddit_columns.keys())].rename(columns=reddit_columns)
    
    combined_df = pd.concat([twitter_std, reddit_std], ignore_index=True)
    print(f"📊 Total combinado: {len(combined_df)} posts")
    
elif twitter_df is not None:
    combined_df = twitter_df.copy()
    print("📊 Usando apenas dados do Twitter")
    
elif reddit_df is not None:
    combined_df = reddit_df.copy()
    print("📊 Usando apenas dados do Reddit")
    
else:
    print("❌ Nenhum dado encontrado para análise")

if combined_df is not None:
    # Converter timestamp para datetime
    combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])
    print(f"📅 Período dos dados: {combined_df['timestamp'].min()} a {combined_df['timestamp'].max()}")

## 📊 Análise Exploratória Básica

Vamos explorar as características básicas dos dados coletados.

In [ ]:
# Estatísticas descritivas
if combined_df is not None:
    print("📈 ESTATÍSTICAS DESCRITIVAS")
    print("="*50)
    
    # Informações gerais
    print(f"📊 Total de posts: {len(combined_df)}")
    print(f"📱 Plataformas: {combined_df['platform'].value_counts().to_dict()}")
    print(f"📅 Período: {combined_df['timestamp'].min().strftime('%Y-%m-%d %H:%M')} a {combined_df['timestamp'].max().strftime('%Y-%m-%d %H:%M')}")
    
    # Estatísticas de texto
    combined_df['text_length'] = combined_df['text'].str.len()
    print(f"\n📝 COMPRIMENTO DOS TEXTOS:")
    print(f"  Média: {combined_df['text_length'].mean():.1f} caracteres")
    print(f"  Mediana: {combined_df['text_length'].median():.1f} caracteres")
    print(f"  Mínimo: {combined_df['text_length'].min()} caracteres")
    print(f"  Máximo: {combined_df['text_length'].max()} caracteres")
    
    # Estatísticas por plataforma
    print(f"\n🔍 POR PLATAFORMA:")
    for platform in combined_df['platform'].unique():
        subset = combined_df[combined_df['platform'] == platform]
        print(f"  {platform.upper()}:")
        print(f"    Posts: {len(subset)}")
        print(f"    Texto médio: {subset['text_length'].mean():.1f} chars")
        print(f"    Período: {subset['timestamp'].min().strftime('%Y-%m-%d')} a {subset['timestamp'].max().strftime('%Y-%m-%d')}")
    
    # Verificar dados ausentes
    print(f"\n🔍 DADOS AUSENTES:")
    missing_data = combined_df.isnull().sum()
    for col, missing in missing_data.items():
        if missing > 0:
            print(f"  {col}: {missing} ({missing/len(combined_df)*100:.1f}%)")
    
    if missing_data.sum() == 0:
        print("  ✅ Nenhum dado ausente encontrado")
    
    # Mostrar exemplos
    print(f"\n📝 EXEMPLOS DE POSTS:")
    for platform in combined_df['platform'].unique():
        subset = combined_df[combined_df['platform'] == platform]
        if len(subset) > 0:
            example = subset.iloc[0]
            print(f"  {platform.upper()}:")
            print(f"    Texto: {example['text'][:100]}...")
            print(f"    Data: {example['timestamp']}")
            print()
else:
    print("❌ Nenhum dado disponível para análise")